In [23]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import joblib
from sklearn.preprocessing import LabelEncoder

In [12]:
#importing data
X=joblib.load('output/X.joblib')
y=joblib.load('output/y.joblib')

In [13]:
print(X.shape,y.shape)

(12798, 40) (12798,)


In [15]:
ncol = X.shape[1]
encoding_dim = 20

In [16]:
input_dim = Input(shape = (ncol, ))

In [17]:
encoded1 = Dense(3000, activation = 'relu')(input_dim)
encoded2 = Dense(2750, activation = 'relu')(encoded1)
encoded3 = Dense(2500, activation = 'relu')(encoded2)
encoded4 = Dense(2250, activation = 'relu')(encoded3)
encoded5 = Dense(2000, activation = 'relu')(encoded4)
encoded6 = Dense(1750, activation = 'relu')(encoded5)
encoded7 = Dense(1500, activation = 'relu')(encoded6)
encoded8 = Dense(1250, activation = 'relu')(encoded7)
encoded9 = Dense(1000, activation = 'relu')(encoded8)
encoded10 = Dense(750, activation = 'relu')(encoded9)
encoded11 = Dense(500, activation = 'relu')(encoded10)
encoded12 = Dense(250, activation = 'relu')(encoded11)
encoded13 = Dense(encoding_dim, activation = 'relu')(encoded12)

# Decoder Layers
decoded1 = Dense(250, activation = 'relu')(encoded13)
decoded2 = Dense(500, activation = 'relu')(decoded1)
decoded3 = Dense(750, activation = 'relu')(decoded2)
decoded4 = Dense(1000, activation = 'relu')(decoded3)
decoded5 = Dense(1250, activation = 'relu')(decoded4)
decoded6 = Dense(1500, activation = 'relu')(decoded5)
decoded7 = Dense(1750, activation = 'relu')(decoded6)
decoded8 = Dense(2000, activation = 'relu')(decoded7)
decoded9 = Dense(2250, activation = 'relu')(decoded8)
decoded10 = Dense(2500, activation = 'relu')(decoded9)
decoded11 = Dense(2750, activation = 'relu')(decoded10)
decoded12 = Dense(3000, activation = 'relu')(decoded11)
decoded13 = Dense(ncol, activation = 'sigmoid')(decoded12)


In [18]:
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded13)

In [19]:
# Compile the Model
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

In [20]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              123000    
_________________________________________________________________
dense_2 (Dense)              (None, 2750)              8252750   
_________________________________________________________________
dense_3 (Dense)              (None, 2500)              6877500   
_________________________________________________________________
dense_4 (Dense)              (None, 2250)              5627250   
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              4502000   
_________________________________________________________________
dense_6 (Dense)              (None, 1750)              3501

In [24]:
#label Encoding the data
lb=LabelEncoder()
y=lb.fit_transform(y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, target, train_size = 0.9, random_state = seed(2017))